In [ ]:
#Imports
from __future__ import division

import random
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

from IPython.display import Markdown as md

print "done importing"

In [ ]:
##aquire the data with the following global variables

N = 1000             #people participate in the market
start_cash = 100.00 #the starting budget per person
rounds = 500        #investment round that the particpants will partake in

if (N%2):
    print "please keep N even for smooth operations"

def func_bound_exp_p(mean, cap):
    chance = cap+1
    while chance > cap:
        chance = np.random.exponential(mean)
    return chance

def func_redistr_wealth(people, d):
    budget = 0
    N = len(people)
    for i in range(N):
        take = people[i][-1] * d
        people[i][-1] -= take
        budget += take
    
    give = budget/N    
    for i in range(N):
        people[i][-1] += give
    
    return people

def func_type_random(type_r, x):
    '''returns a random p up up to 1
    uniformly with a max
    or '''
    if type_r=='exp':
        return func_bound_exp_p(x, 1)
    else:
        return random.random() * x / 1

def func_inv_round(people, tax=0,
                   vola_type='uni', vola=0.5,
                   inve_type='uni', inve=1,
                   loss='mul'): 
    '''
    a single round of investments applied on the people list, returns the modified version
    '''
    N = len(people)
    sN = range(N) #range of indexes of the people participating
    random.shuffle(sN) #shuffled indexes so a random 50% win (first half of the shuffled indexes), the other 50% loses
    
    # percentage market gain/loss this round, up to 50% win or loss 
    p = func_type_random(vola_type, vola)
    
    profit = 1+p
    
    for cnt, Pidx in enumerate(sN):
        if  cnt == int(N / 2) :
            if loss == 'add':
                profit = 1-p
            else:
                profit = 1/(1+p) #if halfway shuffled list, the rest loses money  
        
        #personal profit is based on how many % of their wealth is invested 
        invested = func_type_random(inve_type, inve)  #if exponential with mean x is desired

        
        #appending the new balance to the list
        people[Pidx].append(people[Pidx][-1]*(1 - invested + invested*profit)) #add new entry to the persons money record
                            
    if tax > 0:
        people = func_redistr_wealth(people, tax)

    return people #return the list of people with new balances

def func_investments(r,
                     title = "",
                     tax=0, loss='add',
                     vola_type='uni', vola=1,
                     inve_type='uni', inve=1):
    '''
    runs r rounds of investments
    
    tax: % wealth distribution - how much % of each persons cash is taken and redistribued. 0 = nothing, 1 is everything.
    
    vola_type:  - 'uni' : uniform distribution of volatility
                - 'exp' : exponential distribution
    
    vola: market volatility - for 'uni' this is the max volatility
                            - for 'exp' this is the mean of volatilty
                            - 0->0% ... 1->100%
    inve is for investment distribution and max/mean
                             
    loss:       - 'mul' loss is quantified as 1/p+1 
                - 'add' loss is quantified as 1-p
    '''
    
    people = [[start_cash] for i in range(N)] #list of participants with their money history in a list (start with just 100)

    for i in range(r):
        people = func_inv_round(people, tax=tax,
                                vola_type=vola_type, vola=vola,
                                inve_type=inve_type, inve=inve,
                                loss=loss)
    
    return [people, title]

def func_plot_wealth_vs_t(p_set):
    '''returns the plot of all investment rounds over time'''
    people, title = p_set
    N = len(people)
    
    fig = go.Figure()
    
    for i in range(N):
        fig.add_trace(go.Scatter(x = range(len(people[i])), y = people[i]))
        
    fig.update_layout(
    showlegend=False,
    title='individual wealth over time  ({})'.format(title),
    xaxis_title='investments rounds (t)',
    yaxis_title='individual wealth (€)')
    
    return fig
    
    

    
people_0 = func_investments(rounds, title="default state", loss='add', tax=0, vola_type='exp', vola=0.3, inve_type='uni', inve=1)
func_plot_wealth_vs_t(people_0).show()


In [ ]:
def func_sorted_time(people, t):
    '''
    takes a time as input and returns the sorted list of wealth from htl
    '''
    N = len(people)
    money_at_t = (sorted([people[i][t] for i in range(N)]))[::-1]
    return money_at_t

def func_powerlaw_t(p_set, t, log=False):
    '''
    creates a (lin-lin or log-log) plot for supplied t
    '''
    people, title = p_set
    N = len(people)
    
    money_at_t = func_sorted_time(people, t)
    fpt_plot = go.Figure()
    fpt_plot.add_trace(go.Scatter(x = range(N), y = money_at_t))
    plot_type = "lin-lin"
    if log:
        fpt_plot.update_layout(xaxis_type="log", yaxis_type="log")
        plot_type = "log-log"
        
    
    fpt_plot.update_layout(
    title='individual wealth sorted at t={} on {}  ({})'.format(t, plot_type, title),
    xaxis_title='nth person (n)',
    yaxis_title='individual wealth (€)')
    
    return fpt_plot
    
    
# plot the lin-lin and log-log plot for the last investment round
func_powerlaw_t(people_0, -1).show()
func_powerlaw_t(people_0, -1, log=True).show()

In [ ]:
def func_polyfit_t(people, t):
    '''
    creates a polyfit for t and returns the [a,b] and [error]
    '''
    N = len(people)
    
    data = func_sorted_time(people, t)
    x = np.array(range(len(data))) + 1 # Array with same size as data, starting from 1 to avoid pesky divide-by-zero errors
    polynomial_coefficients, residuals = np.polyfit(np.log(x),np.log(data), 1,full=True)[:2] # Returns coefficients of a polynomial of degree 1 (just a linear relation) with least square fit to data
    
    return polynomial_coefficients, residuals



#calculate the polyfit for the last investment round 
polycoeffs, error = func_polyfit_t(people_0[0], -1) 
print "set = {} @ t = {}".format(people_0[1], -1)
print "polynomial coefficients [a,b] : {}\n               residual/error : {}".format(polycoeffs, error)

In [ ]:
'''turn the previous polynomial coeffs into a nice latex formula using ipynb markdown and python insertion'''

md("the best fitting power law for the log-log plot of the division of wealth is at the last t is:  \n  \n$$y = e^{} + x^{}$$".format(("{"+str(round(polycoeffs[1],3))+"}"), ("{"+str(round(polycoeffs[0],3))+"}")) )

In [ ]:
def func_plots_with_fit(p_set, t, log = True):
    '''
    plot the data of round t with the best fitting power law
    '''
    people, title = p_set
    N = len(people)
    
    data = func_sorted_time(people, t)
    
    polycoeffs, error = func_polyfit_t(people, t)
    
    polynomial = np.poly1d(polycoeffs) #using poly1d to make a function from the variables gained previously
    
    x = np.array(range(len(data))) + 1 # Array with same size as data, starting from 1 to avoid pesky divide-by-zero errors
    
    log_plot = go.Figure()
    log_plot.add_trace(go.Scatter(x = range(N), y = data, name = "data"))
    log_plot.add_trace(go.Scatter(x = range(N), y = np.exp(polynomial(np.log(x))), name = "powerlaw fit"))
    
    plot_type = "lin-lin"
    if log == True:
        log_plot.update_layout(xaxis_type="log", yaxis_type="log")
        plot_type = "log-log"
    
    log_plot.update_layout(
    title='individual wealth sorted at t={} on {} with trendline  ({})'.format(t, plot_type, title),
    xaxis_title='nth person (n)',
    yaxis_title='individual wealth (€)')
    
    return log_plot
    

#plot the lin-lin and log-log plot for the last investment round
func_plots_with_fit(people_0, -1, log=True).show()

In [ ]:
def func_plot_a_vs_t(p_set):
    '''
    plots the a coefficient of each t's polyfit against t to visualize how powerlaw-y the data is. a=2 consitutes a powerlaw
    '''
    people, title = p_set
    N = len(people)
    
    a_over_time = []

    for r in range(rounds):
        polycoeffs, error = func_polyfit_t(people, r)
        a_over_time.append(-1 * polycoeffs[0])

    alpha_plot = go.Figure()
    alpha_plot.add_trace(go.Scatter(x = range(r), y = a_over_time))
    
    alpha_plot.update_layout(
    title='a coefficient of the polyfit for t  ({})'.format(title),
    xaxis_title='investment round (t)',
    yaxis_title='a coefficient of the trend')
    
    return alpha_plot
    
    
    
func_plot_a_vs_t(people_0).show()

In [ ]:
def func_plot_e_vs_t(p_set):
    '''
    plots the residuals of each t's polyfit against t to visualize the acuracy of our fits
    '''
    people, title = p_set
    N = len(people)
    
    e_over_time = []

    for r in range(rounds):
        polycoeffs, error = func_polyfit_t(people, r)
        e_over_time.append(error[0])

    e_plot = go.Figure()
    e_plot.add_trace(go.Scatter(x = range(r), y = e_over_time))

    e_plot.update_layout(
    title='residuals/error of the polyfit for t  ({})'.format(title),
    xaxis_title='investment round (t)',
    yaxis_title='resisuals of the trend')
    
    return e_plot
    
    
    
func_plot_e_vs_t(people_0).show()

In [81]:
def func_halfwaycash(people, t, info = False):
    '''
    given a t it returns the nth person so that everyone including them own closest to half of the wealth at that t
    using info prints the person in text with a bit of explanation
    '''
    N = len(people)
    
    ls_in = func_sorted_time(people, t)
    half_total_cash = sum(ls_in)/2 #what is halfway
    
    track_cash = 0
    wealthy = 0
    for idx, cash in enumerate(ls_in):
        #if the new itteration is closer to half than the previous
        if (abs(half_total_cash - track_cash) >
            abs(half_total_cash - (track_cash+cash))): 
            track_cash += cash
        else:
            if info:
                print("\n\nthere are {} rich that closest split the graph 50/50 left and right with €{} combined".format(idx, round(track_cash,2)))
                print("the nr. {} most wealthy person has €{}".format(idx, round(ls_in[idx-1],2)))
            return (round(idx/N *100,2))
    return

def func_plot_5050_vs_t(p_set):
    '''
    plots the amount least amount of people owning closest to half the wealth of each t against t to visualize the acuracy of our fits
    '''
    people, title = p_set
    N = len(people)
    
    p50_over_time = []

    for r in range(rounds):
        p50_over_time.append(func_halfwaycash(people, r))

    p50_plot = go.Figure()
    p50_plot.add_trace(go.Scatter(x = range(r), y = p50_over_time))

    p50_plot.update_layout(
    title='percentage of people that possess closest to half of the cumulative wealth  ({})'.format(title),
    xaxis_title='investments rounds (t)',
    yaxis_title='% of people')

    return p50_plot




#halfwaycash(func_powerlaw_t(10), info = True)
func_plot_5050_vs_t(people_0)   

In [ ]:
def func_list_average(lst):
    t_avg = sum(lst) / len(lst)
    return t_avg

def func_plot_avg(p_set):
    people, title = p_set
    rounds = len(people[0])
    
    market = []
    market_alt = []
    for t in range(rounds):
        market.append(func_list_average(func_sorted_time(people, t)))
    
    mkt_plot = go.Figure()
    mkt_plot.add_trace(go.Scatter(x = range(rounds), y = market))
    mkt_plot.add_trace(go.Scatter(x = range(rounds), y = market_alt))
    
    mkt_plot.update_layout(
    title='average wealth per round  ({})'.format(title),
    xaxis_title='investment round (t)',
    yaxis_title='average wealth')
    
    return mkt_plot

func_plot_avg(people_0)

In [82]:
def func_plots_a_vs_t(p_sets):
    alpha_plot = go.Figure()
    
    for p_set in p_sets:
        people, title = p_set
        N = len(people)
    
        a_over_time = []

        for r in range(rounds):
            polycoeffs, error = func_polyfit_t(people, r)
            a_over_time.append(-1 * polycoeffs[0])

        alpha_plot.add_trace(go.Scatter(x = range(r), y = a_over_time, name=title))
    
    alpha_plot.update_layout(
    title='a coefficient of the polyfit for t',
    xaxis_title='investment round (t)',
    yaxis_title='a coefficient of the trend')
    
    return alpha_plot

def func_plots_avg(p_sets):
    mkt_plot = go.Figure()
    
    for p_set in p_sets:
        people, title = p_set
        rounds = len(people[0])

        market = []
        for t in range(rounds):
            market.append(func_list_average(func_sorted_time(people, t)))

        mkt_plot.add_trace(go.Scatter(x = range(rounds), y = market, name=title))
    
    mkt_plot.update_layout(
    title='average wealth per round',
    xaxis_title='investment round (t)',
    yaxis_title='average wealth')
    
    return mkt_plot

def func_plots_5050_vs_t(p_sets):
    p50_plot = go.Figure()
    
    for p_set in p_sets:
        people, title = p_set
        N = len(people)
    
        p50_over_time = []

        for r in range(rounds):
            p50_over_time.append(func_halfwaycash(people, r))

    
        p50_plot.add_trace(go.Scatter(x = range(r), y = p50_over_time, name=title))

    p50_plot.update_layout(
    title='percentage of people that possess closest to half of the cumulative wealth',
    xaxis_title='investments rounds (t)',
    yaxis_title='% of people')

    return p50_plot

In [79]:
# people_1 = func_investments(rounds, tax = 0.00003) #0.00003 so 0.003%
# func_plots_with_fit(people_1, -1)
# func_plot_a_vs_t(people_1)
#func_plot_e_vs_t(people_1)
#func_plot_5050_vs_t(people_1)

rounds = 400
N = 10000

p_0 = func_investments(rounds, title="default state",
                                   loss='add', tax=0,
                                   vola_type='uni', vola=0.4,
                                   inve_type='uni', inve=1)

p_1 = func_investments(rounds, title="smarter % investing",
                                   loss='add', tax=0,
                                   vola_type='uni', vola=0.4,
                                   inve_type='exp', inve=0.2)

p_2 = func_investments(rounds, title="global tax",
                                   loss='add', tax=0.01,
                                   vola_type='uni', vola=0.4,
                                   inve_type='uni', inve=1)

p_3 = func_investments(rounds, title="multiplicative loss",
                                   loss='mul', tax=0,
                                   vola_type='uni', vola=0.4,
                                   inve_type='uni', inve=1)

p_4 = func_investments(rounds, title="extreme volatility",
                                   loss='add', tax=0,
                                   vola_type='uni', vola=1,
                                   inve_type='uni', inve=1)

p = [p_0,p_1,p_2,p_3,p_4]

In [83]:
func_plots_avg(p).show()
func_plots_5050_vs_t(p).show()
func_plots_a_vs_t(p).show()

In [72]:
for p_x in p:
    print "{}\n".format(p_x[1])
    #func_plot_wealth_vs_t(p_x).show()
    func_plot_5050_vs_t(p_x).show()
    #func_plots_with_fit(p_x, -1, log=True).show()
    func_plot_a_vs_t(p_x).show()
    func_plot_avg(p_x).show()
    print "\n\n"
    

func_plot_avg(p).show()


default state






smarter % investing






global tax






multiplicative loss

